# Setup 

## Libraries 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.patches import Ellipse, Polygon 
from matplotlib.animation import FuncAnimation

# Animation Calculations 

## process_keyframes 

In [2]:
# function to take a dictionary of keyframes and create a dataframe from it 
def process_keyframes(keyframes): 

    # initial dataframe 
    dfk = pd.DataFrame() 

    # loop through each frame and create a dataframe for it 
    for frame in keyframes:
        dfk = pd.concat([dfk, pd.DataFrame({
            "TIME": frame['time'], 
            "PARAM": frame['values'].keys(), 
            "VALUE": frame['values'].values() 
        })]) 
    
    # reset the index 
    dfk = dfk.reset_index(drop = True) 

    return dfk 

# some sample keyframes 
keyframes = [
    {
        "time": 0, 
        "values": {
            "xc": 30, 
            "yc": 90 
        }
    }, 
    {
        "time": 1, 
        "values": {
            "yc": 100 
        }
    }
] 

# showcase the function 
dfk = process_keyframes(keyframes) 
dfk 

,TIME,PARAM,VALUE
0,0,xc,30
1,0,yc,90
2,1,yc,100


## params_at_time 

In [3]:
# function to calculate parameter values at a given time 
def params_at_time(t, dfk): 

    # sort by time 
    dfk = dfk.sort_values(["TIME"]) 

    # get the parameter values before 
    df_before = (
        dfk[dfk["TIME"] <= t].groupby("PARAM").last() 
        .rename(columns = {"TIME": "TIME_BEFORE", "VALUE": "VALUE_BEFORE"}) 
    ) 

    # get the parameter values after
    df_after = (
        dfk[dfk["TIME"] > t].groupby("PARAM").first() 
        .rename(columns = {"TIME": "TIME_AFTER", "VALUE": "VALUE_AFTER"})
    ) 

    # merge the two and add the time 
    df_merged = df_before.merge(df_after, on = ["PARAM"], how = "outer").reset_index()  
    df_merged["TIME"] = t 

    # calculate the interpolated value
    def interp_value(row): 

        # if either is na, return the other 
        if pd.isna(row["VALUE_BEFORE"]): 
            return row["VALUE_AFTER"] 
        elif pd.isna(row["VALUE_AFTER"]): 
            return row["VALUE_BEFORE"] 

        # otherwise, do a linear interpolation 
        else: 
            t0 = row["TIME_BEFORE"] 
            t1 = row["TIME_AFTER"] 
            v0 = row["VALUE_BEFORE"] 
            v1 = row["VALUE_AFTER"] 
            return v0 + (v1 - v0) * ((t - t0) / (t1 - t0)) 
    
    # apply the interpolation function 
    df_merged["VALUE"] = df_merged.apply(interp_value, axis = 1) 

    # transform to a dictionary 
    pdict = {row["PARAM"]: row["VALUE"] for _, row in df_merged.iterrows()}

    return pdict 

# showcase the function 
pdict = params_at_time(3, dfk) 
pdict 

{'xc': 30, 'yc': 100}

## keyframes_to_params 

In [4]:
# function to transform keyframes into parameter values at a given time 
def keyframes_to_params(t, keyframes, defaults):

    # if no keyframes, just take defaults 
    if len(keyframes) == 0:
        pdict = defaults 

    # otherwise, process keyframes and fill in defaults 
    else:
        dfk = process_keyframes(keyframes) 
        pdict = params_at_time(t, dfk) 

        # fill in default parameters 
        for key, value in defaults.items(): 
            if key not in pdict: 
                pdict[key] = value 
    
    return pdict 

# showcase the function 
keyframes_to_params(
    t = 3, 
    keyframes = keyframes, 
    defaults = {
        "xc": 0, 
        "yc": 0, 
        "alpha": 1 
    }
) 

{'xc': 30, 'yc': 100, 'alpha': 1}

# Animation Objects 

## PlayAnimation 

In [5]:
# class for each play animation 
class PlayAnimation:

    def __init__(self, ystart = 0, yend = 120):
        self.ystart = ystart 
        self.yend = yend 
        self.pixels_width = 1920 
        self.pixels_height = int(self.pixels_width * ((yend - ystart) / 53.3)) 
        self.dpi = 120 
        self.objects = {}  
    
    # create a figure object based on the sizing 
    def create_figure(self):
        return plt.Figure(
            figsize = (
                self.pixels_width / self.dpi, 
                self.pixels_height / self.dpi
            ), 
            dpi = self.dpi 
        ) 

    # create an axis object based on the sizing 
    def create_axis(self, fig, margin = 5):
        return fig.add_axes([
            margin / self.pixels_width, 
            margin / self.pixels_height, 
            1 - (2 * margin / self.pixels_width), 
            1 - (2 * margin / self.pixels_height)
        ]) 
    
    # add an object to the play 
    def add_object(self, obj):
        self.objects[obj.id] = obj 
    
    # add keyframes for the players 
    def add_keyframes(self, keyframes):
        for k, v in keyframes.items():
            if k in self.objects:
                self.objects[k].keyframes += v 

    # draw the play at the current time 
    def draw_play(self, fig, t = 0): 

        # create a new axis 
        ax = self.create_axis(fig) 

        # draw out the objects 
        for obj in self.objects.values():
            obj.draw(ax, t = t) 

        # Set limits and hide axes 
        ax.set_xlim(0, 53.3)
        ax.set_ylim(self.ystart, self.yend) 
        ax.axis('off') 
    
    # save an image of the play at time t 
    def save_image(self, t = 0, save_file = "output_test.png"):

        # draw the play 
        fig = self.create_figure() 
        self.draw_play(fig, t = t) 
        
        # save the figure output for testing 
        fig.savefig(save_file, dpi = self.dpi, pad_inches = 0)

    # create an animation of the play over time 
    def create_animation(self, end_time, fps = 12, save_file = "output_animation.mp4"):
        
        # calculate the times at each animation frame 
        tlist = np.linspace(0, end_time, int(np.max([np.round(end_time * fps), 1]))) 

        # create the figure and axis 
        fig = self.create_figure() 
        ax = self.create_axis(fig) 

        # create the animation function 
        def animate(i):
            if i % 20 == 1:
                print(f"  Creating frame {i} / {len(tlist)}...") 

            # clear out the previous frame 
            fig.clf() 
                
            # draw the play at time tlist[i] 
            self.draw_play(fig, t = tlist[i]) 
        
        # compile and save the animation 
        ani = FuncAnimation(fig, animate, frames = len(tlist) )
        ani.save(save_file, writer = 'ffmpeg', fps = fps) 

## Field 

### draw_field 

In [6]:
# function to draw the field background 
def draw_field(ax, alpha = 1): 

    # Set the background color to green
    ax.add_patch(plt.Rectangle(
        xy = (0, 0), 
        height = 120, 
        width = 53.3, 
        color = [0.32, 0.56, 0.23, alpha], 
        zorder = 0 
    )) 

    # add rectangles for the endzones 
    for i in [0, 110]:
        ax.add_patch(plt.Rectangle(
            xy = (0, i), 
            height = 10, 
            width = 53.3, 
            color = [0.22, 0.38, 0.16, alpha], 
            zorder = 1  
        )) 

    # draw a line for every 5 yards 
    for i in range(0, 21):
        ax.plot(
            [0, 53.3], 
            [10 + (i * 5)] * 2, 
            color = 'white', 
            linewidth = [4, 2][i % 2], 
            alpha = alpha, 
            zorder = 2 
        ) 

    # add the hash marks 
    for x in [20.65, 32.65]: # left and right hash marks 
        for y in range(0, 20):
            yc = (y * 5) + 10 
            ax.plot(
                [x] * 2, 
                [yc - 0.5, yc + 0.5], 
                color = 'white', 
                linewidth = 2, 
                alpha = alpha, 
                zorder = 2 
            ) 

    # add markers for each yard line 
    for x in [0.5, 20.65, 32.65, 52.8]:
        for yl in range(0, 100):
            ax.plot(
                [x - 0.5, x + 0.5], 
                [yl + 10] * 2, 
                color = 'white', 
                linewidth = 2, 
                alpha = alpha, 
                zorder = 2 
            ) 

    # add the numbers 
    for i, txt in enumerate(["1 0", "2 0", "3 0", "4 0", "5 0", "4 0", "3 0", "2 0", "1 0"]):
        for x in [8, 53.3 - 8]:
            ax.text(
                x = x, 
                y = (i * 10) + 20, 
                s = txt, 
                color = 'white', 
                fontsize = 44, 
                fontweight = 'bold', 
                horizontalalignment = 'center', 
                verticalalignment = 'center', 
                rotation = -90 if x < 26.65 else 90, 
                alpha = 0.65 * alpha, 
                zorder = 2 
            ) 

### FieldObject 

In [7]:
# class to represent the field background 
class FieldObject:

    def __init__(self, id = "field"):
        self.id = id 
        self.keyframes = [] 

        # default parameters 
        self.defaults = {
            "alpha": 1 
        }

    # draw the field background
    def draw(self, ax, t = 0):

        # calculate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # draw the field 
        draw_field(ax, alpha = pdict["alpha"]) 

# create the play animation object 
pa = PlayAnimation(ystart = 85, yend = 120) 

# add the field object 
pa.add_object(FieldObject()) 

# test it out 
pa.save_image() 
plt.close() 

## Players 

### draw_player 

In [8]:
# function to draw out a player 
def draw_player(ax, x, y, angle = 0, pcolor = "red", alpha = 1):

    # draw the body ellipse 
    ax.add_patch(Ellipse(
        xy = (x, y), 
        width = 1.5, 
        height = 0.75, 
        angle = -angle, 
        facecolor = pcolor, 
        edgecolor = "black", 
        linewidth = 2,
        zorder = 25, 
        alpha = alpha 
    )) 

    # draw the head ellipse 
    ax.add_patch(Ellipse(
        xy = (x, y), 
        width = 0.5, 
        height = 0.5, 
        angle = -angle, 
        facecolor = pcolor, 
        edgecolor = "black", 
        linewidth = 2,
        zorder = 26, 
        alpha = alpha 
    )) 

    # trapezoid dimensions  
    dw1 = 0.25 
    dw2 = 1   
    dh = 0.75  

    # calculate rotated corners 
    angle_rad = np.radians(-angle) 
    corners = [[-dw1, 0], [-dw2, dh], [dw2, dh], [dw1, 0]]
    rotated_corners = [] 
    for cx, cy in corners:
        new_x = cx * np.cos(angle_rad) - cy * np.sin(angle_rad) + x
        new_y = cx * np.sin(angle_rad) + cy * np.cos(angle_rad) + y
        rotated_corners.append([new_x, new_y])

    # draw the polygon 
    ax.add_patch(Polygon(
        rotated_corners, 
        facecolor = "yellow", 
        edgecolor = None, 
        alpha = 0.65 * alpha, 
        zorder = 24 
    )) 

### PlayerObject 

In [9]:
# class object to define each player 
class PlayerObject:

    def __init__(self, id, pcolor = "red"): 

        # initial parameters 
        self.id = id 
        self.pcolor = pcolor 
        self.keyframes = [] 

        # default parameters 
        self.defaults = {
            "xc": 0, 
            "yc": 0, 
            "angle": 0, 
            "alpha": 1 
        } 

    # draw the player at time t 
    def draw(self, ax, t = 0): 

        # calculate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # draw the player 
        draw_player(
            ax = ax, 
            x = pdict["xc"], 
            y = pdict["yc"], 
            angle = pdict["angle"], 
            pcolor = self.pcolor, 
            alpha = pdict["alpha"] 
        )

# create the play animation object with the field 
pa = PlayAnimation(ystart = 85, yend = 120) 
pa.add_object(FieldObject()) 

# add some players 
pa.add_object(PlayerObject(id = "1", pcolor = "blue")) 
pa.add_object(PlayerObject(id = "2", pcolor = "red")) 

# add keyframes for the players 
pa.add_keyframes({
    "1": [
        {
            "time": 0, 
            "values": {
                "xc": 30, 
                "yc": 90, 
                "angle": 0 
            }
        } 
    ],
    "2": [
        {
            "time": 0, 
            "values": {
                "xc": 35, 
                "yc": 94, 
                "angle": 240 
            }
        }
    ]
}) 

# draw and save the play 
pa.save_image() 
plt.close() 

## Routes 

### create_routes_df 

In [10]:
# function to create a dataframe from the given routes 
def create_routes_df(routes): 

    # initial dataframe 
    dfr = pd.DataFrame() 

    # loop through each route 
    for i, route in enumerate(routes):

        # create a dataframe for the route 
        dfr = pd.concat([dfr, pd.DataFrame({
            "TIME": [((frame - 1) / route["fps"]) + route["start_time"] for frame in route["path"]["frame_id"]],
            "XC": route["path"]["xc"], 
            "YC": route["path"]["yc"] 
        })], ignore_index=True) 
    
    # reset the index 
    dfr = dfr.sort_values("TIME").reset_index(drop = True) 

    # get the previous values 
    dfr["PREV_XC"] = dfr["XC"].shift(1)
    dfr["PREV_YC"] = dfr["YC"].shift(1)
    dfr["PREV_TIME"] = dfr["TIME"].shift(1) 

    return dfr 

# sample routes data 
routes = [
    {
        "start_time": 0, 
        "fps": 10, 
        "path": {
            "frame_id": [1, 2, 3, 4], 
            "xc": [1, 2, 3, 4], 
            "yc": [2, 4, 6, 8] 
        }
    }
] 

# test the function 
dfr = create_routes_df(routes) 
dfr 

,TIME,XC,YC,PREV_XC,PREV_YC,PREV_TIME
0,0.0,1,2,NaN,NaN,NaN
1,0.1,2,4,1.0,2.0,0.0
2,0.2,3,6,2.0,4.0,0.1
3,0.3,4,8,3.0,6.0,0.2


### calc_route_line 

In [11]:
# function to calculate a line at the current time 
def calc_route_line(t, dfr): 

    # loop through each row and add to the arrays 
    xlist = [] 
    ylist = [] 
    for i, row in dfr.iterrows():

        # if time is before or at the current time, add the point 
        if row["TIME"] <= t:
            xlist.append(row["XC"]) 
            ylist.append(row["YC"]) 

        # if time is after the current time, interpolate and break 
        elif row["PREV_TIME"] < t and pd.notna(row["PREV_TIME"]):
            t0 = row["PREV_TIME"] 
            t1 = row["TIME"] 
            x0 = row["PREV_XC"] 
            x1 = row["XC"] 
            y0 = row["PREV_YC"] 
            y1 = row["YC"] 
            x_interp = x0 + (x1 - x0) * ((t - t0) / (t1 - t0)) 
            y_interp = y0 + (y1 - y0) * ((t - t0) / (t1 - t0)) 
            xlist.append(x_interp) 
            ylist.append(y_interp) 
            break 
    
    return xlist, ylist

# test the function 
calc_route_line(0.38, dfr)

([np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)],
 [np.float64(2.0), np.float64(4.0), np.float64(6.0), np.float64(8.0)])

### RouteObject 

In [12]:
# class for player routes/paths 
class RouteObject:

    def __init__(self, id, routes = [], rcolor = "blue"): 

        # initial parameters 
        self.id = id 
        self.rcolor = rcolor 
        self.keyframes = [] 
        self.routes = routes

        # default parameters 
        self.defaults = { 
            "alpha": 1 
        } 
    
    # draw the route at time t 
    def draw(self, ax, t = 0): 

        # calulate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # calculate route path at time t 
        dfr = create_routes_df(self.routes) 
        xlist, ylist = calc_route_line(t, dfr) 

        # draw the route
        ax.plot(
            xlist, 
            ylist, 
            linewidth = 4, 
            color = self.rcolor, 
            alpha = self.defaults["alpha"], 
            zorder = 15 
        ) 

# create the play animation with the field 
pa = PlayAnimation(ystart = 85, yend = 120) 
pa.add_object(FieldObject()) 

# add a route object 
pa.add_object(RouteObject(
    id = "route_1", 
    routes = [
        {
            "start_time": 0, 
            "fps": 10, 
            "path": {
                "frame_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
                "xc": [20, 21, 22, 23, 24, 24, 24, 23, 22, 21], 
                "yc": [90, 91, 92, 93, 94, 95, 96, 97, 98, 99] 
            }
        }
    ], 
    rcolor = "blue"
)) 

# draw and save the play 
pa.save_image(t = 0.85) 
plt.close() 

## Quick Demo

In [13]:
# create the play animation with the field 
pa = PlayAnimation(ystart = 85, yend = 120) 
pa.add_object(FieldObject()) 

# add some players 
pa.add_object(PlayerObject(id = "1", pcolor = "blue")) 
pa.add_object(PlayerObject(id = "2", pcolor = "red")) 

# add keyframes for the players 
pa.add_keyframes({
    "1": [
        {
            "time": 0, 
            "values": {
                "xc": 22, 
                "yc": 90, 
                "angle": -80 
            }
        }, 
        {
            "time": 2, 
            "values": {
                "xc": 17, 
                "yc": 95, 
                "angle": -20 
            }
        } 
    ], 
    "2": [
        {
            "time": 0, 
            "values": {
                "xc": 25, 
                "yc": 100, 
                "angle": -170 
            }
        }, 
        {
            "time": 2, 
            "values": {
                "xc": 18, 
                "yc": 96, 
                "angle": -100 
            }
        } 
    ]
}) 

# add a route for the blue player 
pa.add_object(RouteObject(
    id = "route_1", 
    routes = [
        {
            "start_time": 0, 
            "fps": 0.5, 
            "path": {
                "frame_id": [1, 2], 
                "xc": [22, 17], 
                "yc": [90, 95] 
            }
        }
    ], 
    rcolor = "blue"
)) 

# create and save the animation 
pa.create_animation(
    end_time = 2.5, 
    fps = 12, 
    save_file = "test_animation.mp4"
) 
plt.close() 

  Creating frame 1 / 30...
  Creating frame 21 / 30...


# Add Play Data 

In [ ]:
import os 

folder_path = os.environ.get("NFL_DATA_PATH")



# game_id = 2023121700 
# play_id = 2553 



17833


,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
72382,2023121700,56,True,56097,1,right,35,Carrington Valentine,6-0,194,...,Defensive Coverage,41.29,12.62,0.03,0.03,121.55,252.51,10,40.490002,4.43
72383,2023121700,56,True,56097,2,right,35,Carrington Valentine,6-0,194,...,Defensive Coverage,41.32,12.60,0.03,0.03,120.87,253.96,10,40.490002,4.43
72384,2023121700,56,True,56097,3,right,35,Carrington Valentine,6-0,194,...,Defensive Coverage,41.35,12.59,0.03,0.04,119.15,256.12,10,40.490002,4.43
72385,2023121700,56,True,56097,4,right,35,Carrington Valentine,6-0,194,...,Defensive Coverage,41.42,12.57,0.04,0.33,114.59,257.23,10,40.490002,4.43
72386,2023121700,56,True,56097,5,right,35,Carrington Valentine,6-0,194,...,Defensive Coverage,41.45,12.57,0.30,3.15,108.10,253.59,10,40.490002,4.43


## Play Metadata 

In [24]:
# read in the play metadata
df_meta = pd.read_csv(f"{folder_path}/supplementary_data.csv") 

df_meta.head() 

C:\Users\rjfis\AppData\Local\Temp\ipykernel_19760\3027871378.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta = pd.read_csv(f"{folder_path}/supplementary_data.csv")


,game_id,season,week,game_date,game_time_eastern,home_team_abbr,visitor_team_abbr,play_id,play_description,quarter,...,team_coverage_type,penalty_yards,pre_penalty_yards_gained,yards_gained,expected_points,expected_points_added,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,home_team_win_probability_added,visitor_team_win_probility_added
0,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,3461,(10:46) (Shotgun) J.Goff pass deep left to J.R...,4,...,COVER_2_ZONE,NaN,18,18,-0.664416,2.945847,0.834296,0.165704,-0.081149,0.081149
1,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,461,(7:30) J.Goff pass short right to J.Reynolds t...,1,...,COVER_6_ZONE,NaN,21,21,1.926131,1.345633,0.544618,0.455382,-0.029415,0.029415
2,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1940,(:09) (Shotgun) J.Goff pass incomplete deep ri...,2,...,COVER_2_ZONE,NaN,0,0,0.281891,-0.081964,0.771994,0.228006,0.000791,-0.000791
3,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1711,"(:45) (No Huddle, Shotgun) P.Mahomes pass deep...",2,...,COVER_2_ZONE,NaN,26,26,3.452352,2.342947,0.663187,0.336813,0.041843,-0.041843
4,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1588,(1:54) (Shotgun) P.Mahomes pass incomplete dee...,2,...,COVER_4_ZONE,NaN,0,0,1.921525,-0.324035,0.615035,0.384965,0.000061,-0.000061


## Play Details 

In [25]:
df_plays = pd.read_csv(f"{folder_path}//train//input_2023_w15.csv") 
df_plays = df_plays.loc[df_plays["game_id"] == 2023121700]
print(len(df_plays.index))

df_plays.head() 

17833


,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
72382,2023121700,56,True,56097,1,right,35,Carrington Valentine,6-0,194,...,Defensive Coverage,41.29,12.62,0.03,0.03,121.55,252.51,10,40.490002,4.43
72383,2023121700,56,True,56097,2,right,35,Carrington Valentine,6-0,194,...,Defensive Coverage,41.32,12.60,0.03,0.03,120.87,253.96,10,40.490002,4.43
72384,2023121700,56,True,56097,3,right,35,Carrington Valentine,6-0,194,...,Defensive Coverage,41.35,12.59,0.03,0.04,119.15,256.12,10,40.490002,4.43
72385,2023121700,56,True,56097,4,right,35,Carrington Valentine,6-0,194,...,Defensive Coverage,41.42,12.57,0.04,0.33,114.59,257.23,10,40.490002,4.43
72386,2023121700,56,True,56097,5,right,35,Carrington Valentine,6-0,194,...,Defensive Coverage,41.45,12.57,0.30,3.15,108.10,253.59,10,40.490002,4.43


## PlayDataInput 

In [32]:
# class object for handling the input data on a given play 
class PlayDataInput:

    def __init__(self, game_id, play_id, df_meta, df_plays):
        self.game_id = game_id 
        self.play_id = play_id 

        # filter the metadata for the given play 
        self.play_meta = df_meta.loc[
            (df_meta["game_id"] == game_id) & 
            (df_meta["play_id"] == play_id)
        ].reset_index(drop = True).to_dict(orient = "records")[0] 

        # filter the dataframe for the given play 
        dfp = df_plays.loc[
            (df_plays["game_id"] == game_id) & 
            (df_plays["play_id"] == play_id)
        ].reset_index(drop = True) 

        # transform to match our positioning format 
        dfp["xc"] = 53.3 - dfp["y"] 
        dfp["yc"] = dfp["x"] 
        dfp["angle"] = dfp["o"] - 90 

        # flip to the other direction if the home team is on offense 
        if (self.play_meta["possession_team"] == self.play_meta["home_team_abbr"]):
            dfp["xc"] = 53.3 - dfp["xc"] 
            dfp["yc"] = 120 - dfp["yc"] 
            dfp["angle"] = (dfp["angle"] + 180) % 360 

        # subset columns and add to an attribute 
        self.dfp = dfp[["nfl_id", "player_side", "frame_id", "xc", "yc", "angle"]] 
    
    # function to get data for a specific player 
    def get_player_data(self, player_id):
        return self.dfp.loc[self.dfp["nfl_id"] == player_id].reset_index(drop = True) 

    # get unique players 
    def get_unique_players(self): 
        return self.dfp[["nfl_id", "player_side"]].drop_duplicates() 
    
    # subset frames by a given min/max 
    def subset_player_frames(self, player_id, min_frame = None, max_frame = None):

        # filter for the given player
        df_subset = self.get_player_data(player_id) 

        # subset by min (if given)
        if min_frame is not None:
            df_subset = df_subset.loc[df_subset["frame_id"] >= min_frame]
        
        # subset by max (if given) 
        if max_frame is not None:
            df_subset = df_subset.loc[df_subset["frame_id"] <= max_frame]
        
        return df_subset.reset_index(drop = True) 
    
    # create keyframes for a given player 
    def create_player_keyframes(self, player_id, start_time = 0, fps = 10, min_frame = None, max_frame = None):

        # get the player data 
        df_player = self.subset_player_frames(player_id, min_frame = min_frame, max_frame = max_frame) 

        # get the first frame 
        frame_start = df_player["frame_id"].min() 

        # iterate through each frame and create keyframes 
        keyframes = [] 
        for i, row in df_player.iterrows():
            keyframes.append({
                "time": start_time + ((row["frame_id"] - frame_start) / fps), 
                "values": {
                    "xc": row["xc"], 
                    "yc": row["yc"], 
                    "angle": row["angle"]
                }
            }) 
        
        return keyframes 
    
    # create the route definition for a given player 
    def create_route_definition(self, player_id, start_time = 0, fps = 10, min_frame = None, max_frame = None):

        # get the player data 
        df_player = self.subset_player_frames(player_id, min_frame = min_frame, max_frame = max_frame) 

        # create the route definition 
        route_def = {
            "start_time": start_time, 
            "fps": fps, 
            "path": {
                "frame_id": df_player["frame_id"].tolist(), 
                "xc": df_player["xc"].tolist(), 
                "yc": df_player["yc"].tolist() 
            }
        } 

        return route_def 
    
    # calculate the total time of the play (before the pass) 
    def calc_play_time(self, fps = 10):
        frames = self.dfp["frame_id"].unique().tolist() 
        return len(frames) / fps 
    
    # calculate the window bounds of the play 
    def calc_play_window(self, padding = 5): 
        return [
            max(self.dfp["yc"].min() - padding, 0), 
            min(self.dfp["yc"].max() + padding, 120) 
        ]


pdi = PlayDataInput(game_id = 2023121700, play_id = 2553, df_meta = df_meta, df_plays = df_plays) 

# pdi.get_player_data(player_id = 55915).head() 
pdi.subset_player_frames(player_id = 55915, min_frame = 10, max_frame = 13) 

# pdi.create_player_keyframes(player_id = 55915, min_frame = 10, max_frame = 13) 

# pdi.get_unique_players() 

,nfl_id,player_side,frame_id,xc,yc,angle
0,55915,Offense,10,37.79,94.20,351.25
1,55915,Offense,11,37.85,94.78,358.93
2,55915,Offense,12,37.91,95.40,7.39
3,55915,Offense,13,37.96,96.07,8.68


In [28]:
pdi.play_meta 

{'game_id': 2023121700,
 'season': 2023,
 'week': 15,
 'game_date': '12/17/2023',
 'game_time_eastern': '13:00:00',
 'home_team_abbr': 'GB',
 'visitor_team_abbr': 'TB',
 'play_id': 2553,
 'play_description': '(4:26) (Shotgun) J.Love pass deep right to J.Reed for 17 yards, TOUCHDOWN.',
 'quarter': 3,
 'game_clock': '04:26',
 'down': 3,
 'yards_to_go': 14,
 'possession_team': 'GB',
 'defensive_team': 'TB',
 'yardline_side': 'TB',
 'yardline_number': 17,
 'pre_snap_home_score': 10,
 'pre_snap_visitor_score': 20,
 'play_nullified_by_penalty': 'N',
 'pass_result': 'C',
 'pass_length': 17,
 'offense_formation': 'EMPTY',
 'receiver_alignment': '3x2',
 'route_of_targeted_receiver': 'CORNER',
 'play_action': False,
 'dropback_type': 'SCRAMBLE',
 'dropback_distance': 5.8899998664856,
 'pass_location_type': 'OUTSIDE_RIGHT',
 'defenders_in_the_box': 6,
 'team_coverage_man_zone': 'ZONE_COVERAGE',
 'team_coverage_type': 'COVER_4_ZONE',
 'penalty_yards': nan,
 'pre_penalty_yards_gained': 17,
 'yards_

## setup_play_showcase 

In [33]:
# function to setup an animation for a given play 
def setup_play_showcase(game_id, play_id, df_meta, df_plays): 

    # import the play data 
    # pdi = PlayDataInput(game_id = 2023121700, play_id = 2553, df_plays = df_plays, flip_y = True) 
    pdi = PlayDataInput(
        game_id = game_id, 
        play_id = play_id, 
        df_meta = df_meta, 
        df_plays = df_plays 
    ) 

    # get some info about the play 
    df_players = pdi.get_unique_players() 
    pbounds = pdi.calc_play_window() 

    # create the play animation with the field 
    pa = PlayAnimation(ystart = pbounds[0], yend = pbounds[1]) 
    pa.add_object(FieldObject()) 

    # colors by defense/offense 
    colors = {
        "Offense": "blue",
        "Defense": "red" 
    }

    # add some players 
    for i, row in df_players.iterrows():
        player_id = row["nfl_id"] 
        pa.add_object(PlayerObject(id = f"{player_id}", pcolor = colors[row["player_side"]])) 

        # add keyframes for the players 
        keyframes = pdi.create_player_keyframes(player_id = player_id) 
        pa.add_keyframes({f"{player_id}": keyframes}) 

        # add a route for the player 
        route_def = pdi.create_route_definition(player_id = player_id) 
        pa.add_object(RouteObject(
            id = f"{player_id}_route", 
            routes = [route_def], 
            rcolor = colors[row["player_side"]]
        )) 
    
    return pa, pdi 

# create the play 
pa, pdi = setup_play_showcase(
    game_id = 2023121700, 
    play_id = 2553, 
    df_meta = df_meta, 
    df_plays = df_plays
) 

# test the image at a given time 
pa.save_image(t = 7) 

# # create and save the animation 
# pa.create_animation(
#     end_time = 7, 
#     fps = 12, 
#     save_file = "test_animation.mp4"
# ) 

# close out the plot 
plt.close() 

# Testing Examples 

## Save Screenshots 